# Sample for InferenceService Explainer

The notebook shows how to use KFServing SDK to create InferenceService with explainer, predictor.

In [59]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2ExplainerSpec
from kfserving import V1alpha2AlibiExplainerSpec
from kfserving import V1alpha2SKLearnSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements
from kubernetes import client, config
import kubernetes.client
import os
import requests 


## Define InferenceService with Explainer

Add predictor and explainer on the endpoint spec

In [ ]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            min_replicas=1,
                            sklearn=V1alpha2SKLearnSpec(
                              storage_uri='gs://seldon-models/sklearn/moviesentiment',
                              resources=V1ResourceRequirements(
                                  requests={'cpu': '100m', 'memory': '1Gi'},
                                  limits={'cpu': '100m', 'memory': '1Gi'}))),
                          explainer=V1alpha2ExplainerSpec(
                            min_replicas=1, 
                            alibi=V1alpha2AlibiExplainerSpec(
                              type='AnchorText',
                              resources=V1ResourceRequirements(
                                  requests={'cpu': '100m', 'memory': '2Gi'},
                                  limits={'cpu': '100m', 'memory': '2Gi'}))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='moviesentiment', namespace='kubeflow'),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))


## Create InferenceService

Call KFServingClient to create InferenceService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(isvc)


## Check the InferenceService

In [60]:
KFServing.get('moviesentiment', namespace='kubeflow', watch=True, timeout_seconds=120)


NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
moviesentiment       True                   100                 http://moviesentiment-predict.kubeflow.example.com


## Predict the sentiment for the sentence

In [61]:
api_instance = kubernetes.client.CoreV1Api(kubernetes.client.ApiClient())
service = api_instance.read_namespaced_service("istio-ingressgateway", "istio-system", exact='true')
cluster_ip = service.status.load_balancer.ingress[0].ip
print(cluster_ip)

34.67.11.82


In [62]:
url = "http://" + cluster_ip + "/v1/models/moviesentiment:predict"
headers = {'Host': 'moviesentiment.kubeflow.example.com'}
response = requests.post(url, '{"instances":["This is a bad book ."]}', headers=headers)
print(response.content)
response = requests.post(url, '{"instances":["This is a good book ."]}', headers=headers)
print(response.content)

b'{"predictions": [0]}'
b'{"predictions": [1]}'


## Explain the Prediction

In [58]:
url = "http://" + cluster_ip + "/v1/models/moviesentiment:explain"
headers = {'Host': 'moviesentiment.kubeflow.example.com'}
response = requests.post(url, '{"instances":["This is a bad book ."]}', headers=headers)
print(response.content)
response = requests.post(url, '{"instances":["This is a good book ."]}', headers=headers)
print(response.content)

b'{"names": ["bad"], "precision": 1.0, "coverage": 0.5007, "raw": {"feature": [3], "mean": [1.0], "precision": [1.0], "coverage": [0.5007], "examples": [{"covered": [["This is a bad book UNK"], ["UNK is UNK bad book UNK"], ["UNK is UNK bad book ."], ["This UNK UNK bad book UNK"], ["This is UNK bad UNK ."], ["UNK UNK UNK bad book ."], ["UNK is a bad UNK UNK"], ["UNK UNK a bad UNK ."], ["UNK UNK a bad book UNK"], ["UNK is UNK bad book ."]], "covered_true": [["UNK is UNK bad UNK UNK"], ["UNK is UNK bad UNK UNK"], ["UNK is UNK bad book UNK"], ["This UNK UNK bad book UNK"], ["UNK UNK a bad book ."], ["This is UNK bad UNK UNK"], ["UNK UNK UNK bad UNK UNK"], ["This is UNK bad UNK ."], ["This is UNK bad UNK ."], ["This UNK a bad UNK ."]], "covered_false": [], "uncovered_true": [], "uncovered_false": []}], "all_precision": 0, "num_preds": 1000101, "names": ["bad"], "positions": [10], "instance": "This is a bad book .", "prediction": 0}}'
b'{"names": ["good"], "precision": 1.0, "coverage": 0.500

## Delete the InferenceService

In [ ]:
KFServing.delete('moviesentiment', namespace='kubeflow')
